# Deep Q-Learning for Lunar Landing

## Part 0 - Installing the required packages and importing the libraries

### Installing Gymnasium

In [11]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!pip install swig
!pip install gymnasium[box2d]

In [12]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128

Looking in indexes: https://download.pytorch.org/whl/cu128


In [13]:
import torch
print(torch.cuda.is_available())

True


### Importing the libraries

In [14]:
import os #for os
import random #for random numbers
import numpy as np #for arrays
import torch # to train agent w pytorch
import torch.nn as nn #neural network module
import torch.optim as optim # optimal module
import torch.nn.functional as F #functions pre made for training
import torch.autograd as autograd # for stochastic gradient descent
from torch.autograd import Variable #training
from collections import deque, namedtuple #training

## Part 1 - Building the AI

### Creating the architecture of the Neural Network

In [15]:
class NeuNet(nn.Module): #inherits from nn.module

  def __init__(self, state_size, action_size, seed=42):
    super(NeuNet, self).__init__()
    self.seed = torch.manual_seed(seed)
    self.fc1 = nn.Linear(state_size, 64) #Input Layer
    self.fc2 = nn.Linear(64, 64) #Hidden Layer
    self.fc3 = nn.Linear(64, action_size) #Output Layer

  def forward(self, state):
    x = self.fc1(state)
    x = F.relu(x) #rectifier activation function from torch.nn.functional
    #Gone from first fully conected layer to the second
    x = self.fc2(x)
    x = F.relu(x)
    return self.fc3(x) #output layer



*   fc1 is input layer. Number can be anything however, post testing, 64 is the most optimal in cases for lunar landings in gymnasium
*   fc2 is hidden layer
*   fc3 is output layer. action size is 4
*   DEF FORWARD






## Part 2 - Training the AI

### Setting up the environment

In [16]:
import gymnasium as gym
env = gym.make('LunarLander-v3') # The Lunar Lander environment was upgraded to v3
state_shape = env.observation_space.shape #Vector
state_size = env.observation_space.shape[0] #current state of env
number_actions = env.action_space.n #Number of actions
print('State shape: ', state_shape)
print('State size: ', state_size)
print('Number of actions: ', number_actions)

State shape:  (8,)
State size:  8
Number of actions:  4


### Initializing the hyperparameters

In [17]:
learning_rate = 5e-4 #derived from experimentation
minibatch_size = 100
gamma = 0.99 #discount factor
replay_buffer_size = int(1e5) #no. of experiences
tau = 1e-3 #Interpolation parameter

### Implementing Experience Replay

In [18]:
class ReplayMemory(object):

  def __init__(self, capacity):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #if exterior device(gpu) is present. uses that hardware to process
    self.capacity = capacity #total size of memory size
    self.memory = [ ]

  def push(self, event): #appends event and removes oldest event if memory is full
    self.memory.append(event)
    if len(self.memory) > self.capacity:
      del self.memory[0]

  def sample(self, batch_size):
    exp = random.sample(self.memory, k = batch_size)
    state = torch.from_numpy(np.vstack([e[0] for e in exp if e is not None])).float().to(self.device) #states converted to tensors and float values and send to gpu or cpu
    action = torch.from_numpy(np.vstack([e[1] for e in exp if e is not None])).long().to(self.device) #same as states but long integers
    rewards = torch.from_numpy(np.vstack([e[2] for e in exp if e is not None])).float().to(self.device)
    next_state = torch.from_numpy(np.vstack([e[3] for e in exp if e is not None])).float().to(self.device)
    dones = torch.from_numpy(np.vstack([e[4] for e in exp if e is not None]).astype(np.uint8)).float().to(self.device)
    return state, next_state, action, rewards, dones

### Implementing the DQN class

In [19]:
class Agent():

  def __init__(self, state_size, action_size):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #if exterior device(gpu) is present. uses that hardware to process
    self.state_size = state_size
    self.action_size = action_size
    self.local_qnetwork = NeuNet(state_size, action_size).to(self.device)
    self.target_qnetwork = NeuNet(state_size, action_size).to(self.device)
    self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr = learning_rate)
    self.memory = ReplayMemory(replay_buffer_size)
    self.t_step = 0

  def step(self, state, action, reward, next_state, done):
    self.memory.push((state, action, reward, next_state, done))
    self.t_step = (self.t_step + 1) % 4
    if self.t_step == 0:
      if len(self.memory.memory) > minibatch_size: #self.memory.memory second memory is attribute. while self.memory is the instance of the memory class
        exp_local = self.memory.sample(minibatch_size) #samples 100 experiences from the memory
        self.learn(exp_local, gamma)

  def act(self, state, epsilon = 0.):
    state = torch.from_numpy(state).float().unsqueeze(0).to(self.device) #set as torch tensor and an extra variable to the vector to show batch number
    self.local_qnetwork.eval()
    with torch.no_grad():
      action_values = self.local_qnetwork(state)
    self.local_qnetwork.train()
    if random.random() > epsilon: #epsilon greedy action selection policy
      return np.argmax(action_values.cpu().data.numpy())
    else:
      return random.choice(np.arange(self.action_size))

  def learn(self, exp, gamma):
    states,next_states, actions, rewards, dones = exp
    next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
    q_targets = rewards + (gamma * next_q_targets * (1 - dones))
    q_expected = self.local_qnetwork(states).gather(1, actions)
    loss = F.mse_loss(q_expected, q_targets)
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()
    self.soft_update(self.local_qnetwork, self.target_qnetwork, tau)

  def soft_update(self, local_model, target_model, tau):
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
      target_param.data.copy_(tau * local_param.data + (1.0 - tau) * target_param.data)







### Initializing the DQN agent

In [20]:
agent = Agent(state_size, number_actions)

### Training the DQN agent

In [21]:
number_ep = 2000
max_num_timesteps_per_ep = 1000
epsilon_start = 1.0
epsilon_end = 0.01
epsilon_decay = 0.995
epsilon = epsilon_start
scores = deque(maxlen = 100)

for episode in range(1, number_ep + 1):
  state, _ = env.reset() #other information is discarded w underscore
  score = 0
  for t in range(max_num_timesteps_per_ep):
    action = agent.act(state, epsilon)
    next_state, reward, done, _, _ = env.step(action)
    agent.step(state, action, reward, next_state, done)
    state = next_state
    score += reward
    if done:
      break
  scores.append(score)
  epsilon = max(epsilon_end, epsilon_decay*epsilon)
  print('\rEpisode Number {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores)), end="")
  if episode % 100 == 0:
   print('\rEpisode Number {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores)))
  if np.mean(scores) >= 200.0:
      print('\nEnviroment solved in {:d} episodes!Episode Number \tAverage Score: {:.2f}'.format(episode, np.mean(scores)))
      torch.save(agent.local_qnetwork.state_dict(), 'checkpoint charlie')
      break

Episode Number 100	Average Score: -163.49
Episode Number 200	Average Score: -89.583
Episode Number 300	Average Score: -14.87
Episode Number 400	Average Score: 49.071
Episode Number 500	Average Score: 143.34
Episode Number 600	Average Score: 184.36
Episode Number 630	Average Score: 200.13
Enviroment solved in 630 episodes!Episode Number 	Average Score: 200.13


## Part 3 - Visualizing the results

In [22]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'LunarLander-v3')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()

ModuleNotFoundError: No module named 'imageio'